In [1]:
from fetch_data import fetch_data_from_db

sql = """
    SELECT stk_tkr FROM os_stk_info
"""

stk_tkr = fetch_data_from_db(query=sql)

In [2]:
import pandas as pd

stk_tkr = pd.DataFrame(stk_tkr)

tkr_list = stk_tkr['stk_tkr'].to_list()
tkr_list_str = " ".join(tkr_list)

In [3]:
import yfinance as yf

data = yf.download(tkr_list_str, period='1y')

[*********************100%***********************]  712 of 712 completed


In [18]:
data['Adj Close'].head()

,A,AA,AAON,AAPL,ABBV,ABNB,ACA,ACAD,ACB,ACCD,...,YY,Z,ZBH,ZBRA,ZG,ZIP,ZLAB,ZM,ZS,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-13,154.054382,47.587734,77.601158,153.430313,150.658447,116.419998,58.233990,19.180000,0.915,11.15,...,34.778454,44.830002,127.593941,318.950012,44.150002,22.500000,40.389999,76.480003,133.869995,23.355606
2023-02-14,152.499878,48.560329,78.050011,152.782074,149.541229,120.870003,58.233990,19.100000,0.928,12.01,...,34.691483,45.540001,126.258713,318.760010,44.889999,23.219999,37.990002,77.900002,136.190002,23.743210
2023-02-15,151.214432,48.044258,78.758194,154.906265,148.041794,137.009995,59.492134,18.969999,0.975,12.49,...,33.995720,47.490002,126.059418,327.019989,46.660000,23.730000,37.160000,80.769997,142.770004,24.230198
2023-02-16,147.756668,48.401539,78.478905,153.290680,146.542374,139.419998,59.182590,18.709999,0.935,11.88,...,33.531876,47.900002,124.664406,333.440002,47.049999,23.600000,37.119999,77.910004,132.929993,23.385422
2023-02-17,147.736740,46.535736,79.795532,152.133850,148.286804,131.600006,58.393753,19.370001,0.936,11.82,...,32.121029,45.889999,125.521347,322.579987,44.959999,23.809999,37.380001,76.110001,132.350006,23.285740


In [4]:
df_list = []

for tkr in tkr_list:
    df = pd.DataFrame()
    
    df.index = data['Adj Close'].index
    df["close"] = data['Adj Close'][tkr]
    df["stk_tkr"] = tkr
    
    df_list.append(df)

In [5]:
stk_price_df = pd.concat(df_list)

In [6]:
stk_price_df.reset_index(inplace=True)
stk_price_df.rename(columns={"Date": "date"}, inplace=True)
stk_price_df.head()

,date,close,stk_tkr
0,2022-08-15,132.998550,A
1,2022-08-16,131.876160,A
2,2022-08-17,141.332077,A
3,2022-08-18,139.027679,A
4,2022-08-19,136.693512,A


In [7]:
stk_price_df.to_csv("./data_before_db_insert/os_stk_price.csv")